In [1]:
from pyspark.sql.session import SparkSession
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
from pyspark.sql.types import *
from pyspark.sql.functions import concat,concat_ws
from pyspark.sql.functions import *
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
# nltk.download('punkt')
# nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
# nltk.download('wordnet')
import pandas as pd
import sklearn as sk
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
findspark.init()
findspark.find()

'C:\\spark\\spark-3.5.0-bin-hadoop3'

In [39]:
product_path='D:/PROJECTS/FASHIONPASS/Database/product.csv'
tagcloud_path='D:/PROJECTS/FASHIONPASS/Database/tag_cloud.csv'
collection_category_tag_path='D:/PROJECTS/FASHIONPASS/Database/collection_category_tag.csv'
product_tag_path='D:/PROJECTS/FASHIONPASS/Database/product_tag.csv'
features_path='D:/PROJECTS/FASHIONPASS/Database/spars_dense_features.csv'
collection_category_path='D:/PROJECTS/FASHIONPASS/Database/collection_category.csv'
clothing_collection_path='D:/PROJECTS/FASHIONPASS/Database/cloting_collection_category_10_1_2023.csv'
product_cloth_collection_path='D:/PROJECTS/FASHIONPASS/Database/product_cloth_cloection_tag_v2_10_1_2023.csv'#D:/PROJECTS/FASHIONPASS/Database/product_cloth_cloection_tag.csv'

In [40]:
spark = SparkSession.builder \
    .appName("PySpark MySQL Example") \
    .config("spark.driver.extraClassPath", "C:/spark/spark-3.5.0-bin-hadoop3/jars/mysql-connector-java-5.1.46.jar") \
    .getOrCreate()



In [41]:
Product_df=spark.read.format('csv').option('header',True) \
.load(product_path)

Tagcloud_df=spark.read.format('csv').option('header',True) \
.load(tagcloud_path)


Collection_Category_tag_df=spark.read.format('csv').option('header',True) \
.load(collection_category_tag_path)

Product_tag_df=spark.read.format('csv').option('header',True) \
.load(product_tag_path)


features_df=spark.read.format('csv').option('header',True) \
.load(features_path)


Collection_Category_df=spark.read.format('csv').option('header',True)\
.load(collection_category_path)

clothing_collection_df=spark.read.format('csv').option('header',True)\
.load(clothing_collection_path)

product_cloth_collection_tag_df=spark.read.format('csv').option('header',True)\
.load(product_cloth_collection_path)

In [27]:
Product_df.show()

+---+----------+--------------------+-------------------+--------------------+
|_c0|product_id|       product_title|         created_at|   product_thumbnail|
+---+----------+--------------------+-------------------+--------------------+
|  0|         1|2-WAY STRETCH TWI...|2018-08-28 00:39:16|2-way-stretch-twi...|
|  1|         2|     3 DISC NECKLACE|2018-04-24 04:26:29|3discnecklace-gor...|
|  2|         3|     3 DISC NECKLACE|2018-04-24 06:24:33|3discnecklace-gor...|
|  3|         4|A LITTLE BIT OF S...|2018-02-13 23:25:38|alittlebitofsomet...|
|  4|         5|    ABBEY MINI DRESS|2018-06-28 03:58:56|abbey-mini-dress-...|
|  5|         6|ABBY OFF SHOULDER...|2018-08-07 23:05:01|abby-off-shoulder...|
|  6|         7|ABBY OFF SHOULDER...|2018-08-09 21:56:29|abby-off-shoulder...|
|  7|         8|ABBY OFF SHOULDER...|2018-08-09 22:09:12|abby-off-shoulder...|
|  8|         9|      ACE MINI DRESS|2018-12-05 05:25:42|ace-mini-dress-fo...|
|  9|        10|      ACE SUNGLASSES|2017-08-22 10:4

In [28]:
product_cloth_collection_tag_df.show()

+----------+--------------------------+------------------------+-----------------------+----------------------------------+-------------------------------------------------+------------------------------------------+------------------------------------+-------------------------------+-------------------------------+--------------------------------+----------------------------------+---------------------------------+----------------------------------+-----------------------------------+----------------------------------+-------------------+--------------+---------+--------------+-------------+-------------------+-------------+--------------+----------+-------------+----------------+----------------+------------+--------------+-------------+------------+-----------+---------------+-------------------+-----------+------------------+---------------+-----------+-----------------------------+---------------+---------------+-------------------------------------+-------------------------+-----

In [42]:
product_cloth_collection_tag_df=product_cloth_collection_tag_df.select(['product_id','tag_string'])

In [43]:
product_cloth_collection_tag_df.show()

+----------+--------------------+
|product_id|          tag_string|
+----------+--------------------+
|       829|                 ...|
|      1436|    Activewear & ...|
|      3414|                 ...|
|      2162|                 ...|
|      6731|                B...|
|      1159|                B...|
|      2294|                 ...|
|      4937|                B...|
|      6240|                 ...|
|       296|                 ...|
|      8304|    Activewear & ...|
|      2088|                 ...|
|      7711|                 ...|
|      2136|                 ...|
|      3606|                 ...|
|      4821|                B...|
|       467|                 ...|
|      7762|                 ...|
|      1090|                B...|
|      4032|                 ...|
+----------+--------------------+
only showing top 20 rows



In [9]:
stemmer=PorterStemmer()
lemmatizer=WordNetLemmatizer()

In [10]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')
def stemming_tokenizer(str_input):
    words = re.sub(r'[^a-zA-Z]{2,}', '', str_input).lower().split()
    words = [stemmer.stem(word) for word in words]
    return words

In [44]:
product_tag_string=[ string[0].lower() for string in product_cloth_collection_tag_df.select('tag_string').collect()]
product_tag_id=[ id[0] for id in product_cloth_collection_tag_df.select('product_id').collect()]

In [45]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('bert-base-nli-mean-tokens')

# Encode sentences
embeddings = model.encode(product_tag_string, convert_to_tensor=True)

In [46]:
cos_sim = util.cos_sim(embeddings, embeddings)

cos_sim

tensor([[1.0000, 0.8443, 0.7701,  ..., 0.5004, 0.0752, 0.4624],
        [0.8443, 1.0000, 0.8980,  ..., 0.4739, 0.0618, 0.4243],
        [0.7701, 0.8980, 1.0000,  ..., 0.5289, 0.0393, 0.4988],
        ...,
        [0.5004, 0.4739, 0.5289,  ..., 1.0000, 0.2598, 0.8794],
        [0.0752, 0.0618, 0.0393,  ..., 0.2598, 1.0000, 0.1792],
        [0.4624, 0.4243, 0.4988,  ..., 0.8794, 0.1792, 1.0000]])

In [47]:
all_product_combination = {'cosin_score':[] , 'Product_A':[] , 'Product_B':[]}
for i in range(len(cos_sim)-1):
    for j in range(i+1, len(cos_sim)):
        all_product_combination['cosin_score'].append(float(cos_sim[i][j]))
        all_product_combination['Product_A'].append(product_tag_id[i])
        all_product_combination['Product_B'].append(product_tag_id[j])

In [48]:
recomendation_df=pd.DataFrame(all_product_combination)

In [52]:
recomendation_df.to_csv('D:/PROJECTS/FASHIONPASS/cosin_similarity_registory/cosine_similarity_cloection_tag_v1_1_10_2023.csv')

In [51]:
product_title='GOOD CURVE STRAIGHT'
selected_product_id=Product_df.filter(Product_df['product_title']==product_title).select('product_id').collect()[0][0]
recomendation_df_product=recomendation_df[recomendation_df['Product_A']==selected_product_id]
link=[]
for index,row in recomendation_df_product.iterrows():
    if row['cosin_score']>=.90:
        id=row['Product_B']
        # link.append(int(id))
        # product_thumbnail=Product_df.filter(Product_df.product_id==id).select('product_thumbnail').collect()
        try:
            # print('https://fashionpass.s3.us-west-1.amazonaws.com/products/'+(product_thumbnail[0][0]))
             link.append(id)
        except:
            print('valid string')    
print(','.join(link))

5226,6904,7814,5857,7824,4974,8126,4698,8330,7861,6700,3084,7639,6173,8842,7138,8710,2396,785,6420,6024,6675,2877,3909,202,3161,8368,8336,6210,8090,2845,5907,6180,8314,7878,2798,1354,8334,3790,2441,4407,549,8594,7534,3436,4815,7758,7052,7893,7998,3159,7753,8301,8611,5277,7147,4535,8302,7894,8593,4571,4900,8733,7109,8753,5184,8097,5244,8093,7642,7658,8573,7895,3769,7143,6906,7142,8162,7139,8143,2344,8545,8094,44,6175,8250,8663,7640,8285,8253,8473,3961,8709,7826,7760,4406,7808,1674,4568,7817,1057,916,8698,4927,8096,3319,7757,7060,8655,8434,7617,6287,8163,8134,7535,6902,7759,7393,5225,6932,7106,8784,7511,7144,6177,7011,3320,3435,1450,3025,7929,7619,8666,4813,8335,8207,2549,7991,2333,5424,2697,5422,7993,4328,5245,8544,7503,7913,6199,769,8264,8729,6197,7146,7986,6178,8251,8542,6763,575,7653,4803,8624,1614,5038,3271,2001,4816,3007,7592,6610,8369,2137,8105,7928,1913,7660,3767,6740,7914,8541,8587,129,4629,2029,7531,1662,3396,2800,8588,8728,3198,7990,6992,7862,5957


In [80]:
recomendation_df_product

,cosin_score,Product_A,Product_B
17068902,0.398466,7641,4827
17068903,0.389854,7641,5683
17068904,0.660179,7641,7731
17068905,0.713733,7641,3301
17068906,0.406955,7641,7220
...,...,...,...
17073414,0.113454,7641,8507
17073415,0.522113,7641,5885
17073416,0.470148,7641,8133
17073417,0.441442,7641,8078
